### Importig dependencies

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import sqlite3
import pickle

import time
import re
import nltk
from nltk.corpus import stopwords

### Reading the data

In [25]:
conn = sqlite3.connect(r'C:\Users\DELL\Desktop\Applied AI\SVM\Self project\foodreview-svm\database.sqlite')

filtered_data = pd.read_sql_query("""SELECT * FROM Reviews where Score!=3""",conn)
conn.close()

In [26]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [27]:
filtered_data.shape

(525814, 10)

#### Classifying data into positive & negative reviews

In [28]:
filtered_data['Score'] = filtered_data['Score'].map(lambda x: 1 if x>3 else 0)

In [29]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


### Removing the duplicate features

In [30]:
filtered_data["Time"].nunique()

3157

In [31]:
filtered_data[(filtered_data["UserId"] == "A74SHV5ZD3RLT") & (filtered_data['Summary'] == 'The price is right')] # Example of duplicate data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
8166,8935,B0007A0AP8,A74SHV5ZD3RLT,"R. Ellis ""Bobby""",15,15,1,1303862400,The price is right,We have a little Maltese that we spoil to no e...
162162,175817,B0014DUUFC,A74SHV5ZD3RLT,"R. Ellis ""Bobby""",15,15,1,1303862400,The price is right,We have a little Maltese that we spoil to no e...
494170,534267,B0007A0AOY,A74SHV5ZD3RLT,"R. Ellis ""Bobby""",15,15,1,1303862400,The price is right,We have a little Maltese that we spoil to no e...
504729,545770,B001E5E1C8,A74SHV5ZD3RLT,"R. Ellis ""Bobby""",15,15,1,1303862400,The price is right,We have a little Maltese that we spoil to no e...


In [32]:
final = filtered_data.drop_duplicates(subset = ["UserId", "Text"])
final.shape

(363859, 10)

In [33]:
# Helpfulness Denominator should greater than or equal to Helpfulness Numerator
final = final[final["HelpfulnessDenominator"] >= final["HelpfulnessNumerator"]]
final.shape

(363857, 10)

In [34]:
# Checking number of possitive & negative reviews
final["Score"].value_counts()

1    306779
0     57078
Name: Score, dtype: int64

### Cleaning text reviews

In [35]:
# Identifying HTML tags from Text
i = 0
for sen in final ['Text'].values:
    if(len(re.findall('<.*?>', sen))):
        print(sen,"\n\n")
        i += 1
    if i == 5:
        break

I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service! 


Twizzlers, Strawberry my childhood favorite candy, made in Lancaster Pennsylvania by Y & S Candies, Inc. one of the oldest confectionery Firms in the United States, now a Subsidiary of the Hershey Company, the Company

In [36]:
# Listing the stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

{"it's", 'off', 'y', 'did', 'doing', 'above', 'out', 'at', 'while', 'should', 'o', 'or', "mustn't", 'until', 've', 'i', 'too', 'only', 'of', "isn't", 'so', "aren't", 'are', 'than', 'me', 'each', "you're", "doesn't", 'yourself', 'which', 't', 'where', 'there', 'not', 'his', 'were', 'was', 'no', "you'll", 'all', 'few', 'the', 'mightn', 'been', 'him', 'their', 're', 'he', 'yourselves', 'm', 'before', 'whom', 'from', 'once', 'has', 'hasn', 'being', 'ourselves', 'needn', "shouldn't", 'about', 'hers', 's', 'nor', 'further', 'over', "mightn't", 'ain', 'we', 'am', 'up', 'down', 'my', 'this', 'now', 'you', 'didn', 'shouldn', "hasn't", 'ours', "weren't", 'aren', 'couldn', 'as', 'herself', 'very', 'themselves', 'for', 'mustn', "don't", 'will', 'its', 'between', 'weren', 'be', 'is', 'myself', 'why', 'wouldn', "she's", 'own', 'isn', 'your', 'it', 'in', 'if', 'through', "won't", 'wasn', 'other', 'because', 'having', 'how', "that'll", 'she', 'does', 'such', 'don', 'to', "you've", 'those', 'by', 'shan

In [37]:
# sample stemming
snow = nltk.stem.SnowballStemmer('english')
print(snow.stem('gracefully'))

grace


In [38]:
# Combining together
def data_cleaning(series):
    i = 0
    final_string = []
    list_of_sent = []
    cleanr = re.compile('<.*?>')
    
    for sent in series.values:
        filtered_sent = []
        sent = re.sub(cleanr, ' ', sent)
        sent = re.sub('[^a-zA-Z0-9\n]', ' ', sent)
        sent = re.sub('\s+',' ', sent)
        sent = sent.lower()
        
        for word in sent.split():
            if word not in stop_words and len(word)>2:
                word = snow.stem(word)
                filtered_sent.append(word)
        list_of_sent.append(filtered_sent)
        string = ' '.join(filtered_sent)
        final_string.append(string)
        i+=1
        
    return final_string, list_of_sent

In [39]:
start = time.time()
final_string, list_of_sent = data_cleaning(final['Text'])
end = time.time()
print("Time takes in seconds =", end - start)

Time takes in seconds = 404.6606695652008


In [40]:
final['CleanedText']=final_string
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,bought sever vital can dog food product found ...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arriv label jumbo salt peanut peanut a...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,confect around centuri light pillowi citrus ge...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,look secret ingredi robitussin believ found go...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffi great price wide assort yummi taff...


In [41]:
conn = sqlite3.connect('final.sqlite')
c = conn.cursor()
final.to_sql('Reviews', conn, if_exists = 'replace', index = False)
conn.close()

In [42]:
with open('list_of_sent.pkl', 'wb') as pickle_file:
    pickle.dump(list_of_sent_sent, pikle_file)

NameError: name 'list_of_sent_sent' is not defined